## Assigment: Neural network basics

Soft deadline: 16.09.18 at 23.59

Hard deadline: 18.09.18 at 23.59

For this task I intentionally provide no boilerplate code, because very puprpose of this task is getting you comforatable with basic code template for desiging NNs in pytorch. I higly recommend you to revisit all the last seminar materials.

#### Task

* Implement simple **fully-convolutional** neural architecture for classification. Make sure it is small enought to run on your home machine.
* Provide dataset visulization.
* Provide train/test split and validation

#### Requirements

* Architecture should derive from `torch.nn.Module`
* Use `torch.utils.data.Dataset` and `torch.utils.data.DataLoader`. But if you manage co simplify this step using dataset `torchivision`, I will only encourage you.
* Implement at least one data transformer, but make sure it is useful for classification task.
* Use FashionMNIST dataset https://github.com/zalandoresearch/fashion-mnist
* Make sure you can fix random seed for all components of your code to make experiments reproducible
* Since you architecure should be fully-convolutional, make sure it does not depend on input size.

In [61]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.backends.cudnn.deterministic=True

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(1, 16, 3),
                                   #  nn.MaxPool2d(3, stride=1)
                                   nn.BatchNorm2d(16),
                                   nn.LeakyReLU())

        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 3),
                                  # nn.MaxPool2d(3, stride=1)
                                   nn.BatchNorm2d(32),
                                   nn.LeakyReLU())
        
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 3),
                                   nn.BatchNorm2d(64),
                                   nn.LeakyReLU(),
                                   nn.MaxPool2d(2))
        
        self.conv4 = nn.Sequential(nn.Conv2d(64, 16, 3),
                                   nn.BatchNorm2d(16),
                                   nn.LeakyReLU(),
                                   nn.MaxPool2d(2))

        self.conv5 = nn.Sequential(nn.Conv2d(16, 10, 3),
                                   nn.BatchNorm2d(10),
                                   nn.LeakyReLU(),
                                   nn.MaxPool2d(2))


    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        output = F.max_pool2d(x, kernel_size=x.size()[2:])
        return output

def one_hot(labels, num_classes=10):

    y = torch.eye(num_classes, dtype=torch.long) 
    return y[labels] 


def train(epoch_num, model, optimizer, loss_func, train_loader):
    model.train()
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        
        x, y = Variable(x), Variable(y)
        optimizer.zero_grad()
        
        y_ = model(x)
        y_ = y_.squeeze(3)
        y_ = y_.squeeze(2)

        loss = loss_func(y_, y)
        loss.backward()
        optimizer.step()
        
        pred = y_.data.max(1)[1]
        correct = pred.eq(y.data).cpu().sum()

        acc = float(correct) / len(pred)
        
        if batch_idx % 450 == 0:
            print(f'Train Epoch: {epoch_num+1} [{ batch_idx * len(x)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: { loss.item():.6f} acc={acc:.3f}')
            
def test(epoch_num, model, loss_func, test_loader):
    with torch.no_grad():
        model.eval()
        test_loss = 0
        correct = 0
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            
            y_ = model(x)
            y_ = y_.squeeze(3)
            y_ = y_.squeeze(2)
            
            test_loss += loss_func(y_, y).item()
            pred = y_.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(y.data).cpu().sum()

        acc = 100. * float(correct) / len(test_loader.dataset)

        test_loss /= len(test_loader.dataset)
        print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{ len(test_loader.dataset)}'
              f' ({acc:.0f}%)\n')


def main():
    
    n_epoches = 20
    model = MyModel().to(device)
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    train_loader = DataLoader(
        datasets.FashionMNIST(root=".", train=True, download=True, transform=transforms.Compose([
            
            transforms.RandomHorizontalFlip(),
            transforms.RandomAffine(20),
            transforms.RandomCrop(26),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=64, shuffle=True, num_workers=4)

    test_loader = DataLoader(
        datasets.FashionMNIST(root='.', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)),
        ])), 
        batch_size=64, shuffle=True, num_workers=4)

    for i_epoch in range(n_epoches):
        train(i_epoch, model, optimizer, loss_func, train_loader)
        test(i_epoch,model,loss_func, test_loader)


if __name__ == "__main__":
    main()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.461066 acc=0.094
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.952362 acc=0.688
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.573722 acc=0.906

Test set: Average loss: 0.0106, Accuracy: 8000/10000 (80%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.656304 acc=0.859
Train Epoch: 2 [28800/60000 (48%)]	Loss: 0.707872 acc=0.781
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.493176 acc=0.859

Test set: Average loss: 0.0081, Accuracy: 8291/10000 (83%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.776980 acc=0.734
Train Epoch: 3 [28800/60000 (48%)]	Loss: 0.624430 acc=0.781
Train Epoch: 3 [57600/60000 (96%)]	Loss: 0.479863 acc=0.844

Test set: Average loss: 0.0074, Accuracy: 8389/10000 (84%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.428938 acc=0.875
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.461656 acc=0.844
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.371216 acc=0.891

Test set: Average loss: 0.0070, Accuracy: 8420/10000 (84%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.848792 acc=0.734
T